# Google drive

In [3]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/My Drive/fruit-and-vegetable-image-recognition'

# Get the absolute path of the train folder
abspath_train = '/content/drive/My Drive/fruit-and-vegetable-image-recognition/train/'

# Get the absolute path of the test folder
abspath_test = '/content/drive/My Drive/fruit-and-vegetable-image-recognition/test/'

# Get the absolute path of the test folder
abspath_validation = '/content/drive/My Drive/fruit-and-vegetable-image-recognition/validation/'

Mounted at /content/drive


# Download and loading Fruits and Vegetables Image Recognition dataset

In [4]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

train_dir = '/content/drive/My Drive/fruit-and-vegetable-image-recognition/train/'
data = tf.keras.preprocessing.image_dataset_from_directory(train_dir)

test_dir = '/content/drive/My Drive/fruit-and-vegetable-image-recognition/test/'
data = tf.keras.preprocessing.image_dataset_from_directory(test_dir)

val_dir = '/content/drive/My Drive/fruit-and-vegetable-image-recognition/validation/'
data = tf.keras.preprocessing.image_dataset_from_directory(val_dir)


Found 3115 files belonging to 36 classes.
Found 359 files belonging to 36 classes.
Found 351 files belonging to 36 classes.


In [5]:
# Create an ImageDataGenerator and do Image Augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.15,
                                   shear_range=0.15,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(224, 224))


validation_generator = valid_datagen.flow_from_directory(val_dir,
                                                         batch_size=32,
                                                         class_mode='categorical',
                                                         target_size=(224, 224))

Found 3115 images belonging to 36 classes.
Found 351 images belonging to 36 classes.


In [6]:
num_classes = len(data.class_names)
print('.... Number of Classes : {0} ....'.format(num_classes))

.... Number of Classes : 36 ....


In [7]:
#Defing a function to see images
def show_img(data):
    plt.figure(figsize=(15,15))
    for images, labels in data.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            ax.imshow(images[i].numpy().astype("uint8"))
            ax.axis("off")

#Plotting the images in dataset
show_img(data)

KeyboardInterrupt: ignored

<Figure size 1080x1080 with 0 Axes>

# Training Model

In [ ]:
# Download Inception v3 weights to local machine
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

pre_trained_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights=None)

# Load Inception v3 weights into model
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model.load_weights(local_weights_file)

--2022-02-27 23:59:40--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.128, 142.250.99.128, 74.125.199.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   229MB/s    in 0.4s    

2022-02-27 23:59:41 (229 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
for layer in pre_trained.layers:
    layer.trainable = False

In [ ]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_376 (Conv2D)            (None, 111, 111, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_378 (Batch  (None, 111, 111, 32  96         ['conv2d_376[0][0]']             
 Normalization)                 )                                                      

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(36, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_376 (Conv2D)            (None, 111, 111, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_378 (Batch  (None, 111, 111, 32  96         ['conv2d_376[0][0]']             
 Normalization)                 )                                                           

In [14]:
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = validation_generator.n // validation_generator.batch_size

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch = STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps = STEP_SIZE_VALID,
                    epochs = 30,
                    verbose = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/30
 2/97 [..............................] - ETA: 2:40 - loss: 4.1559 - accuracy: 0.0000e+00 

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


11/97 [==>...........................] - ETA: 15:59 - loss: 3.7026 - accuracy: 0.0284

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


97/97 [==============================] - 1194s 12s/step - loss: 2.5032 - accuracy: 0.3386 - val_loss: 0.9300 - val_accuracy: 0.6938
Epoch 2/30
97/97 [==============================] - 182s 2s/step - loss: 1.2034 - accuracy: 0.6572 - val_loss: 0.3729 - val_accuracy: 0.8750
Epoch 3/30
97/97 [==============================] - 183s 2s/step - loss: 0.8178 - accuracy: 0.7532 - val_loss: 0.3790 - val_accuracy: 0.8656
Epoch 4/30
97/97 [==============================] - 183s 2s/step - loss: 0.6452 - accuracy: 0.7918 - val_loss: 0.2114 - val_accuracy: 0.9344
Epoch 5/30
97/97 [==============================] - 185s 2s/step - loss: 0.5294 - accuracy: 0.8388 - val_loss: 0.2355 - val_accuracy: 0.9281
Epoch 6/30
97/97 [==============================] - 186s 2s/step - loss: 0.4544 - accuracy: 0.8579 - val_loss: 0.2068 - val_accuracy: 0.9406
Epoch 7/30
97/97 [==============================] - 186s 2s/step - loss: 0.3693 - accuracy: 0.8842 - val_loss: 0.2001 - val_accuracy: 0.9563
Epoch 8/30
97/97 [====

In [2]:
import pandas as pd
pd.DataFrame(history.history)[['accuracy','val_accuracy']].plot()
plt.title("Accuracy")
plt.show()

NameError: ignored

In [ ]:
pd.DataFrame(history.history)[['loss','val_loss']].plot()
plt.title("Loss")
plt.show()